# **vLife Virtusa**

# **Genome Analysis and finding frequency of Protien using BioPython**

I am going to use Biopython to analyze the established sequence of the caronavirus. For documentation look here: BioPython

Some facts regarding the virus itself: COVID-19

It is of the (+)ssRNA classification of viruses, which means it is a single stranded virus that can be directly translated into protein.
The actual virus is called SARS-CoV-2, Covid-19 is the name for the respiratory disease it causes.

**Usecase Description**

Genome Analysis is performed with BioPython, The FASTA DNA sequence or Protein sequence is considered as an input.

**DNA sequence input - **

DNA sequence is transcribed (T (for Thymine) is replaced with U (for Uracil)) to mRNA which is then translated to Proteins, then functional protiens are extracted from the protein sequences(which is protien of length>20).
We analyse the largest protein sequence and we obtain the following:
1. The Aminoacid frequency is then obtained from the largest protein sequence found.
2. IsoElectric point - The isoelectric point, is the pH at which a molecule carries no net electrical charge or is electrically neutral in the statistical mean
3. Aromaticity - In organic chemistry, aromaticity is a property of cyclic, planar structures with a ring of resonance bonds that gives increased stability compared to other geometric or connective arrangements with the same set of atoms.
4. Molecular weight - The molecular mass is the mass of a given molecule: it is measured in daltons. Different molecules of the same compound may have different molecular masses because they contain different isotopes of an element.
5. Secondary structure composition.

**Protein sequence input**

Protein sequence is considered as an input and then functional protiens are extracted from the protein sequences(which is protien of length>20).
We analyse the largest protein sequence and we obtain the following:
1. The Aminoacid frequency is then obtained from the largest protein sequence found.
2. IsoElectric point - The isoelectric point, is the pH at which a molecule carries no net electrical charge or is electrically neutral in the statistical mean
3. Aromaticity - In organic chemistry, aromaticity is a property of cyclic, planar structures with a ring of resonance bonds that gives increased stability compared to other geometric or connective arrangements with the same set of atoms.
4. Molecular weight - The molecular mass is the mass of a given molecule: it is measured in daltons. Different molecules of the same compound may have different molecular masses because they contain different isotopes of an element.
5. Secondary structure composition.

**Dataset**
The Dataset used in the usecase is FASTA format DNA or Protein of COVID-19 isolate.

**Dataset source**
The dataset source - https://bigd.big.ac.cn/gwh/browse/virus/coronaviridae

In [ ]:
import pandas as pd
import os
from flask import Flask, render_template, request
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from werkzeug.utils import secure_filename
import re
###New section ############

import Bio
import operator
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, generic_rna
from Bio.Blast import NCBIWWW
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO
from Bio import pairwise2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


amino_acids_mapping = {v: k for k, v in Bio.SeqUtils.IUPACData.protein_letters_3to1.items()}


We use BioPython packages for the analysis

# **DNA INPUT**

In [6]:
class genome():
    def genome_frequency(dnaseq):
        sample_DNAseq = dnaseq
        sample_DNA = sample_DNAseq.seq
        sample_DNA_a = sample_DNA[0:100]
        bb = ('A', 'T', 'C', 'G', 'Y', 'W', 'U')
        for x in str(sample_DNA_a):
            if x not in bb:
                result_value = False
                print("You have entered a Protein file")
                break
            else:
                result_value = True
                print("You have entered a DNA file")
                break
        if result_value == True:
            mRNA_sample = sample_DNA.transcribe()

            Amino_Acid = mRNA_sample.translate()
            lenofamino = len(Amino_Acid)
            lenmRna = len(mRNA_sample)
            from Bio.Data import CodonTable
            condon = CodonTable.unambiguous_rna_by_name['Standard']

            Proteins_sample = Amino_Acid.split('*')

            Protein_dataframe = pd.DataFrame(Proteins_sample)
            Protein_dataframe.describe()

            def conv(item):
                return len(item)

            def to_str(item):
                return str(item)

            Protein_dataframe['sequence_string'] = Protein_dataframe[0].apply(to_str)
            Protein_dataframe['length'] = Protein_dataframe[0].apply(conv)
            Protein_dataframe.rename(columns={0: "sequence of protein"}, inplace=True)

            functional_proteins = Protein_dataframe.loc[Protein_dataframe['length'] >= 20]
            lenfunprotein = len(functional_proteins)

            functional_proteins.describe()
            indexprotein = Protein_dataframe['length'].idxmax()

            for i in Proteins_sample[:]:
                if len(i) < 20:
                    Proteins_sample.remove(i)
            Proteins_sample
            largest_prot = max(Proteins_sample, key=len)
            lenlarget_prot = len(largest_prot)
            indexprot = Proteins_sample.index(largest_prot)

            from Bio.SeqUtils.ProtParam import ProteinAnalysis

            MolecularWeight = []
            amino_freq = []
            aromaticity = []
            IsoElectric = []
            for i in Proteins_sample[:]:
                test = ProteinAnalysis(str(i))
                MolecularWeight.append(test.molecular_weight())
                amino_freq.append(test.count_amino_acids())
                aromaticity.append(test.aromaticity())
                IsoElectric.append(test.isoelectric_point())

            aromaticity_d = aromaticity[indexprot]
            aromaticity_d = str(round(aromaticity_d, 2))
            IsoElectric_d = IsoElectric[indexprot]
            IsoElectric_d = str(round(IsoElectric_d, 2))

            MolecularWeight = pd.DataFrame(data=MolecularWeight, columns=["Molecular Weights"])
            MolecularWeight['Molecular Weights'].max()

            MolecularWeight.idxmax()
            from collections import Counter
            res = Counter(largest_prot)

            alpha_result = []
            listToStr = ' '.join([str(elem) for elem in largest_prot])
            bb = ('A', 'E', 'G', 'K', 'L', 'M')
            for x in listToStr:
                if x in bb:
                    # print(x)
                    alpha_result.append(x)

            alpha_result = ' '.join([str(elem) for elem in alpha_result])

            largest_prot_str = ''.join([str(elem) for elem in largest_prot])

            from Bio.SeqUtils.ProtParam import ProteinAnalysis
            X = ProteinAnalysis(largest_prot_str)
            sec_struc = X.secondary_structure_fraction()
            mol_weight = X.molecular_weight()
            mol_weight = str(round(mol_weight, 2))
            mRNA_sample = str(mRNA_sample)
            Amino_Acid = str(Amino_Acid)
            largest_prot = str(largest_prot)
            return {"Length of RNA ":lenmRna},{"Length of Amino Acids ":lenofamino},{"Length of functional protiens ":lenfunprotein},{"Length of the largest protein ":lenlarget_prot},{"Aromaticity ":aromaticity_d}, {"Isoelectricity ":IsoElectric_d}, {"Frequency of Amino Acids ":res}, {"Secondary Structure composition ":sec_struc}, {"Molecular weight ": mol_weight}
        else:
            sample_prot = dnaseq
            Proteins_sample = sample_prot.seq
            largest_prot = Proteins_sample
            lenlarget_prot = len(largest_prot)
            indexprot = Proteins_sample.index(largest_prot)

            from Bio.SeqUtils.ProtParam import ProteinAnalysis

            MolecularWeight = []
            amino_freq = []
            aromaticity = []
            IsoElectric = []
            for i in Proteins_sample[:]:
                test = ProteinAnalysis(str(i))
                MolecularWeight.append(test.molecular_weight())
                amino_freq.append(test.count_amino_acids())
                aromaticity.append(test.aromaticity())
                IsoElectric.append(test.isoelectric_point())

            aromaticity_d = aromaticity[indexprot]
            aromaticity_d = str(round(aromaticity_d, 2))
            IsoElectric_d = IsoElectric[indexprot]
            IsoElectric_d = str(round(IsoElectric_d, 2))

            MolecularWeight = pd.DataFrame(data=MolecularWeight, columns=["Molecular Weights"])
            MolecularWeight['Molecular Weights'].max()


            MolecularWeight.idxmax()

            from collections import Counter
            res = Counter(largest_prot)

            alpha_result = []
            listToStr = ' '.join([str(elem) for elem in largest_prot])
            bb = ('A', 'E', 'G', 'K', 'L', 'M')
            for x in listToStr:
                if x in bb:
                    # print(x)
                    alpha_result.append(x)

            alpha_result = ' '.join([str(elem) for elem in alpha_result])

            largest_prot_str = ''.join([str(elem) for elem in largest_prot])

            from Bio.SeqUtils.ProtParam import ProteinAnalysis
            X = ProteinAnalysis(largest_prot_str)
            sec_struc = X.secondary_structure_fraction()

            mol_weight = X.molecular_weight()
            mol_weight = str(round(mol_weight, 2))
            largest_prot = str(largest_prot)
            return {"Length of the largest protein ":lenlarget_prot}, {"Aromaticity ":aromaticity_d}, {"Isoelectricity ":IsoElectric_d}, {"Frequency of Amino Acids ":res}, {"Secondary Structure composition ":sec_struc}, {"Molecular weight ": mol_weight}



dna_seq = SeqIO.read("/kaggle/input/testdatagenome/NC_046965.genome.fasta","fasta")
genome.genome_frequency(dna_seq)

You have entered a DNA file


({'Length of RNA ': 28539},
 {'Length of Amino Acids ': 9513},
 {'Length of functional protiens ': 46},
 {'Length of the largest protein ': 2697},
 {'Aromaticity ': '0.12'},
 {'Isoelectricity ': '6.58'},
 {'Frequency of Amino Acids ': Counter({'L': 224,
           'F': 159,
           'K': 155,
           'R': 106,
           'V': 230,
           'G': 154,
           'S': 181,
           'E': 104,
           'A': 160,
           'I': 131,
           'P': 112,
           'T': 165,
           'D': 170,
           'Q': 93,
           'C': 97,
           'N': 164,
           'M': 59,
           'H': 61,
           'Y': 144,
           'W': 28})},
 {'Secondary Structure composition ': (0.3396366332962551,
   0.2265480163144234,
   0.20281794586577678)},
 {'Molecular weight ': '304733.82'})

# **Protein Input**

In [7]:
class genome():
    def genome_frequency(dnaseq):
        sample_DNAseq = dnaseq
        sample_DNA = sample_DNAseq.seq
        sample_DNA_a = sample_DNA[0:100]
        bb = ('A', 'T', 'C', 'G', 'Y', 'W', 'U')
        for x in str(sample_DNA_a):
            if x not in bb:
                result_value = False
                print("You have entered a Protein file")
                break
            else:
                result_value = True
                print("You have entered a DNA file")
                break
        if result_value == True:
            mRNA_sample = sample_DNA.transcribe()

            Amino_Acid = mRNA_sample.translate()
            lenofamino = len(Amino_Acid)
            lenmRna = len(mRNA_sample)
            from Bio.Data import CodonTable
            condon = CodonTable.unambiguous_rna_by_name['Standard']

            Proteins_sample = Amino_Acid.split('*')

            Protein_dataframe = pd.DataFrame(Proteins_sample)
            Protein_dataframe.describe()

            def conv(item):
                return len(item)

            def to_str(item):
                return str(item)

            Protein_dataframe['sequence_string'] = Protein_dataframe[0].apply(to_str)
            Protein_dataframe['length'] = Protein_dataframe[0].apply(conv)
            Protein_dataframe.rename(columns={0: "sequence of protein"}, inplace=True)

            functional_proteins = Protein_dataframe.loc[Protein_dataframe['length'] >= 20]
            lenfunprotein = len(functional_proteins)

            functional_proteins.describe()
            indexprotein = Protein_dataframe['length'].idxmax()

            for i in Proteins_sample[:]:
                if len(i) < 20:
                    Proteins_sample.remove(i)
            Proteins_sample
            largest_prot = max(Proteins_sample, key=len)
            lenlarget_prot = len(largest_prot)
            indexprot = Proteins_sample.index(largest_prot)

            from Bio.SeqUtils.ProtParam import ProteinAnalysis

            MolecularWeight = []
            amino_freq = []
            aromaticity = []
            IsoElectric = []
            for i in Proteins_sample[:]:
                test = ProteinAnalysis(str(i))
                MolecularWeight.append(test.molecular_weight())
                amino_freq.append(test.count_amino_acids())
                aromaticity.append(test.aromaticity())
                IsoElectric.append(test.isoelectric_point())

            aromaticity_d = aromaticity[indexprot]
            aromaticity_d = str(round(aromaticity_d, 2))
            IsoElectric_d = IsoElectric[indexprot]
            IsoElectric_d = str(round(IsoElectric_d, 2))

            MolecularWeight = pd.DataFrame(data=MolecularWeight, columns=["Molecular Weights"])
            MolecularWeight['Molecular Weights'].max()

            MolecularWeight.idxmax()
            from collections import Counter
            res = Counter(largest_prot)

            alpha_result = []
            listToStr = ' '.join([str(elem) for elem in largest_prot])
            bb = ('A', 'E', 'G', 'K', 'L', 'M')
            for x in listToStr:
                if x in bb:
                    # print(x)
                    alpha_result.append(x)

            alpha_result = ' '.join([str(elem) for elem in alpha_result])

            largest_prot_str = ''.join([str(elem) for elem in largest_prot])

            from Bio.SeqUtils.ProtParam import ProteinAnalysis
            X = ProteinAnalysis(largest_prot_str)
            sec_struc = X.secondary_structure_fraction()
            mol_weight = X.molecular_weight()
            mol_weight = str(round(mol_weight, 2))
            mRNA_sample = str(mRNA_sample)
            Amino_Acid = str(Amino_Acid)
            largest_prot = str(largest_prot)
            return {"Length of RNA ":lenmRna},{"Length of Amino Acids ":lenofamino},{"Length of functional protiens ":lenfunprotein},{"Length of the largest protein ":lenlarget_prot},{"Aromaticity ":aromaticity_d}, {"Isoelectricity ":IsoElectric_d}, {"Frequency of Amino Acids ":res}, {"Secondary Structure composition ":sec_struc}, {"Molecular weight ": mol_weight}
        else:
            sample_prot = dnaseq
            Proteins_sample = sample_prot.seq
            largest_prot = Proteins_sample
            lenlarget_prot = len(largest_prot)
            indexprot = Proteins_sample.index(largest_prot)

            from Bio.SeqUtils.ProtParam import ProteinAnalysis

            MolecularWeight = []
            amino_freq = []
            aromaticity = []
            IsoElectric = []
            for i in Proteins_sample[:]:
                test = ProteinAnalysis(str(i))
                MolecularWeight.append(test.molecular_weight())
                amino_freq.append(test.count_amino_acids())
                aromaticity.append(test.aromaticity())
                IsoElectric.append(test.isoelectric_point())

            aromaticity_d = aromaticity[indexprot]
            aromaticity_d = str(round(aromaticity_d, 2))
            IsoElectric_d = IsoElectric[indexprot]
            IsoElectric_d = str(round(IsoElectric_d, 2))

            MolecularWeight = pd.DataFrame(data=MolecularWeight, columns=["Molecular Weights"])
            MolecularWeight['Molecular Weights'].max()


            MolecularWeight.idxmax()

            from collections import Counter
            res = Counter(largest_prot)

            alpha_result = []
            listToStr = ' '.join([str(elem) for elem in largest_prot])
            bb = ('A', 'E', 'G', 'K', 'L', 'M')
            for x in listToStr:
                if x in bb:
                    # print(x)
                    alpha_result.append(x)

            alpha_result = ' '.join([str(elem) for elem in alpha_result])

            largest_prot_str = ''.join([str(elem) for elem in largest_prot])

            from Bio.SeqUtils.ProtParam import ProteinAnalysis
            X = ProteinAnalysis(largest_prot_str)
            sec_struc = X.secondary_structure_fraction()

            mol_weight = X.molecular_weight()
            mol_weight = str(round(mol_weight, 2))
            largest_prot = str(largest_prot)
            return {"Length of the largest protein ":lenlarget_prot}, {"Aromaticity ":aromaticity_d}, {"Isoelectricity ":IsoElectric_d}, {"Frequency of Amino Acids ":res}, {"Secondary Structure composition ":sec_struc}, {"Molecular weight ": mol_weight}



dna_seq = SeqIO.read("/kaggle/input/prot-file/1P9U_B.Protein.faa","fasta")
genome.genome_frequency(dna_seq)


You have entered a Protein file


({'Length of the largest protein ': 302},
 {'Aromaticity ': '0.0'},
 {'Isoelectricity ': '5.24'},
 {'Frequency of Amino Acids ': Counter({'S': 31,
           'G': 31,
           'L': 26,
           'R': 11,
           'K': 12,
           'M': 10,
           'A': 13,
           'Q': 8,
           'P': 8,
           'V': 30,
           'E': 19,
           'C': 6,
           'I': 13,
           'Y': 14,
           'N': 25,
           'W': 3,
           'D': 7,
           'H': 6,
           'T': 16,
           'F': 13})},
 {'Secondary Structure composition ': (0.32781456953642385,
   0.31456953642384106,
   0.22516556291390727)},
 {'Molecular weight ': '33094.13'})

# **Conclusion -** 

The sequence entered is analysed using BioPython packages and the frequency of Amino acids present in the largest protein sequence along with the Aromaticity, Isoelectric point, Molecular weight and secondary structure composition is rendered as the output